In [1]:
import keras
import tensorflow as tf
import keras
from keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim)]
        )
        self.layernom_1 = layers.LayerNormalization()
        self.layernom_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernom_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernom_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim
        })
        return config


2024-12-13 15:12:35.540771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 15:12:35.589340: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 15:12:35.592118: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-13 15:12:35.592125: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [2]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("../../Data/aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(train_dir, batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(val_dir, batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(base_dir / "test", batch_size=batch_size)
text_vectorization = layers.TextVectorization(max_tokens=20000, output_mode="multi_hot")
text_only_train_ds = train_ds.map(lambda x, _: x)

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(max_tokens = max_tokens, output_mode="int",output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

Found 20338 files belonging to 2 classes.


2024-12-13 15:11:28.238464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-13 15:11:28.240083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-13 15:11:28.240105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-12-13 15:11:28.240121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-12-13 15:11:28.240137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Found 4662 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [3]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transform_encoder.keras", save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20


I0000 00:00:1734094542.134767 1058373 service.cc:148] XLA service 0x723e30016dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734094542.134800 1058373 service.cc:156]   StreamExecutor device (0): NVIDIA RTX 4000 Ada Generation, Compute Capability 8.9
2024-12-13 13:55:42.157769: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734094542.369652 1058373 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-13 13:55:42.732370: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_81', 4 bytes spill stores, 4 bytes spill loads

2024-12-13 13:55:42.743440: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_7', 8 bytes 

  2/636 ━━━━━━━━━━━━━━━━━━━━ 49s 78ms/step - accuracy: 0.4219 - loss: 2.0905

2024-12-13 13:55:46.911970: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_compare_reduce_fusion', 148 bytes spill stores, 108 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_12', 68 bytes spill stores, 68 bytes spill loads

I0000 00:00:1734094546.924401 1058373 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


635/636 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5467 - loss: 0.8534

...

636/636 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5469 - loss: 0.8530

2024-12-13 13:56:30.944274: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_compare_reduce_fusion', 148 bytes spill stores, 108 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_11', 68 bytes spill stores, 68 bytes spill loads

2024-12-13 13:56:31.604566: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_4', 68 bytes spill stores, 68 bytes spill loads

2024-12-13 13:56:32.843215: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_7', 8 bytes spill stores, 8 bytes spill loads

2024-12-13 13:56:32.870894: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory 

636/636 ━━━━━━━━━━━━━━━━━━━━ 52s 74ms/step - accuracy: 0.5470 - loss: 0.8527 - val_accuracy: 0.7997 - val_loss: 0.4247
Epoch 2/20


2024-12-13 13:56:33.636935: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_3', 68 bytes spill stores, 68 bytes spill loads



636/636 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.8143 - loss: 0.4072 - val_accuracy: 0.8488 - val_loss: 0.3447
Epoch 3/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 39s 62ms/step - accuracy: 0.8526 - loss: 0.3431 - val_accuracy: 0.8533 - val_loss: 0.3408
Epoch 4/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 39s 62ms/step - accuracy: 0.8651 - loss: 0.3108 - val_accuracy: 0.8486 - val_loss: 0.3409
Epoch 5/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.8765 - loss: 0.2863 - val_accuracy: 0.8492 - val_loss: 0.3550
Epoch 6/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 42s 66ms/step - accuracy: 0.8820 - loss: 0.2703 - val_accuracy: 0.8608 - val_loss: 0.3284
Epoch 7/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 42s 65ms/step - accuracy: 0.8995 - loss: 0.2454 - val_accuracy: 0.8586 - val_loss: 0.3332
Epoch 8/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.9078 - loss: 0.2228 - val_accuracy: 0.8597 - val_loss: 0.3403
Epoch 9/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.9173 - loss: 0.1984 - val_accurac

In [15]:
model = keras.models.load_model("transform_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

/home/raphael/python/latest/.venv/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


777/782 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8609 - loss: 0.3236

2024-12-13 14:14:40.975318: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_7', 8 bytes spill stores, 8 bytes spill loads

2024-12-13 14:14:41.107163: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_10', 76 bytes spill stores, 92 bytes spill loads

2024-12-13 14:14:41.109890: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11', 420 bytes spill stores, 244 bytes spill loads

2024-12-13 14:14:41.214426: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_7', 8 bytes spill stores, 8 bytes spill loads

2024-12-13 14:14:41.238160: I external/local_xla/xla/stream_executor/cud

782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8610 - loss: 0.3236
Test acc: 0.863


2024-12-13 14:14:42.133685: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_4', 68 bytes spill stores, 68 bytes spill loads



In [2]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.positional_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1] # inputs.shape[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_postitions = self.positional_embeddings(positions)
        return embedded_tokens + embedded_postitions
    
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "input_dim": self.input_dim,
            "sequence_length": self.sequence_length
        })
        return config

In [3]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length=sequence_length, input_dim=vocab_size, output_dim=embed_dim)(inputs)
x = TransformerEncoder(embed_dim=embed_dim, dense_dim=dense_dim, num_heads=num_heads)(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics = ["accuracy"])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        5273600   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

2024-12-13 15:13:50.473325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-12-13 15:13:50.474733: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-13 15:13:50.474752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-12-13 15:13:50.474767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-12-13 15:13:50.474780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras", save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)


NameError: name 'int_train_ds' is not defined

In [ ]:

model = keras.models.load_model("full_transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder, "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")